<a href="https://colab.research.google.com/github/Pathway2008/sub_pred/blob/main/sub_col%2B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import pandas as pd
info = pd.read_csv('/content/drive/MyDrive/predict_subcribe/data_info.csv')
sub = pd.read_csv('/content/drive/MyDrive/predict_subcribe/sample_submission.csv')
train = pd.read_csv('/content/drive/MyDrive/predict_subcribe/train.csv')
test = pd.read_csv('/content/drive/MyDrive/predict_subcribe/test.csv')

In [1]:
!pip install autogluon

In [52]:
train = train.drop(['user_id'],axis=1)

In [53]:
from sklearn.preprocessing import LabelEncoder
labels = [col for col in train.columns if train[col].dtype == 'object' and col not in ['user_id']]
label_encoders = {}
for col in labels:
    label_encoders[col] = LabelEncoder()
    train[col] = label_encoders[col].fit_transform(train[col])
    test[col] = label_encoders[col].transform(test[col])

In [54]:
train['harder'] = 0
train.loc[(train['average_time_per_learning_session'] > 167), 'harder'] = 1
test['harder'] = 0
test.loc[(test['average_time_per_learning_session'] > 167), 'harder'] = 1

In [55]:
train['degree'] = train['monthly_active_learning_days'] * train['average_time_per_learning_session']
train['concentrate'] = train['average_time_per_learning_session'] - train['average_login_time']

test['degree'] = test['monthly_active_learning_days'] * test['average_time_per_learning_session']
test['concentrate'] = test['average_time_per_learning_session'] - test['average_login_time']

In [19]:
train['passionate'] = 0
train.loc[(train['average_time_per_learning_session'] > 75) & (train['abandoned_learning_sessions'] <= 3), 'passionate'] = 1

test['passionate'] = 0
test.loc[(test['average_time_per_learning_session'] > 75) & (test['abandoned_learning_sessions'] <= 3), 'passionate'] = 1

In [32]:
train['uninterest'] = 0
train.loc[(train['abandoned_learning_sessions']>=5) & (train['average_time_per_learning_session']< 37), 'uninterest'] = 1

test['uninterest'] = 0
test.loc[(test['abandoned_learning_sessions']>=5) & (test['average_time_per_learning_session']< 37), 'uninterest'] = 1

In [33]:
from autogluon.tabular import TabularDataset, TabularPredictor
train = TabularDataset(train)
test = TabularDataset(test)

In [35]:
fit_parameters = {
    'presets': 'best_quality',
    'num_bag_folds': 7,
    'num_bag_sets': 5,
    'num_stack_levels': 2
}

In [36]:
predictor = TabularPredictor(
    label='target',
    eval_metric='f1_macro',
    sample_weight='balance_weight'
)

predictor.fit(
    train,
    auto_stack=True,
    fit_weighted_ensemble=True,
    presets=fit_parameters,
    num_gpus=1,
    hyperparameter_tune_kwargs='auto'
)

No path specified. Models will be saved in: "AutogluonModels/ag-20231210_091223"
Presets specified: [{'presets': 'best_quality', 'num_bag_folds': 7, 'num_bag_sets': 5, 'num_stack_levels': 2}]
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=7, num_bag_sets=5
Using predefined sample weighting strategy: balance_weight. Evaluation metrics will ignore sample weights, specify weight_evaluation=True to instead report weighted metrics.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231210_091223"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Wed Aug 30 11:19:59 UTC 2023
CPU Count:          8
Memory Avail:       45.48 GB / 51.00 GB (89.2%)
Disk Space Avail:   128.35 GB / 166.77 GB (77.0%)
Train Data Rows:    10000
Train Data Columns: 17
Label Column:       target
AutoGluon infers your prediction

  0%|          | 0/1 [00:00<?, ?it/s]

	Fitting 35 child models (S1F1 - S5F7) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.02%)
Fitted model: LightGBMXT_BAG_L1/T1 ...
	0.5052	 = Validation score   (f1_macro)
	47.13s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: LightGBM_BAG_L1 ...


  0%|          | 0/1 [00:00<?, ?it/s]

	Fitting 35 child models (S1F1 - S5F7) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.02%)
Fitted model: LightGBM_BAG_L1/T1 ...
	0.5227	 = Validation score   (f1_macro)
	45.84s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: RandomForestGini_BAG_L1 ...
	No hyperparameter search space specified for RandomForestGini_BAG_L1. Skipping HPO. Will train one model based on the provided hyperparameters.
Fitted model: RandomForestGini_BAG_L1 ...
	0.4365	 = Validation score   (f1_macro)
	2.76s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: RandomForestEntr_BAG_L1 ...
	No hyperparameter search space specified for RandomForestEntr_BAG_L1. Skipping HPO. Will train one model based on the provided hyperparameters.
Fitted model: RandomForestEntr_BAG_L1 ...
	0.4498	 = Validation score   (f1_macro)
	3.35s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L1 .

...

Fitted model: NeuralNetFastAI_BAG_L3/96b14c35 ...
	0.5662	 = Validation score   (f1_macro)
	240.56s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: XGBoost_BAG_L3 ...


  0%|          | 0/1 [00:00<?, ?it/s]

	Fitting 35 child models (S1F1 - S5F7) | Fitting with ParallelLocalFoldFittingStrategy (2.0 workers, per: cpus=4, gpus=0, memory=0.05%)
Fitted model: XGBoost_BAG_L3/T1 ...
	0.5658	 = Validation score   (f1_macro)
	86.29s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch_BAG_L3 ...
/usr/local/lib/python3.10/dist-packages/ray/air/config.py:803: UserWarning: Setting a `RunConfig.local_dir` is deprecated and will be removed in the future. If you are not using remote storage,set the `RunConfig.storage_path` instead. Otherwise, set the`RAY_AIR_LOCAL_CACHE_DIR` environment variable to control the local cache location.
  warnings.warn(


+----------------------------------------------------------+
| Configuration for experiment     NeuralNetTorch_BAG_L3   |
+----------------------------------------------------------+
| Search algorithm                 SearchGenerator         |
| Scheduler                        FIFOScheduler           |
| Number of trials                 1                       |
+----------------------------------------------------------+

View detailed results here: /content/AutogluonModels/ag-20231210_091223/models/NeuralNetTorch_BAG_L3



Fitted model: NeuralNetTorch_BAG_L3/ca8e60ea ...
	0.5433	 = Validation score   (f1_macro)
	448.14s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBMLarge_BAG_L3 ...
	Fitting 35 child models (S1F1 - S5F7) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.07%)
	0.5663	 = Validation score   (f1_macro)
	53.17s	 = Training   runtime
	0.84s	 = Validation runtime
Fitting model: WeightedEnsemble_L4 ...
	Ensemble Weights: {'CatBoost_BAG_L1/T1': 0.226, 'CatBoost_BAG_L2/T1': 0.177, 'LightGBM_BAG_L3/T1': 0.097, 'XGBoost_BAG_L3/T1': 0.097, 'LightGBMLarge_BAG_L3': 0.097, 'NeuralNetFastAI_BAG_L2/aa854e28': 0.081, 'LightGBMXT_BAG_L3/T1': 0.048, 'NeuralNetFastAI_BAG_L3/96b14c35': 0.048, 'NeuralNetTorch_BAG_L3/ca8e60ea': 0.048, 'CatBoost_BAG_L3/T1': 0.032, 'ExtraTreesEntr_BAG_L1': 0.016, 'NeuralNetFastAI_BAG_L1/5d0f573a': 0.016, 'LightGBM_BAG_L2/T1': 0.016}
	0.581	 = Validation score   (f1_macro)
	16.92s	 = Training   runtime
	0.02s	 = Vali

In [26]:
predictor.feature_importance(data=train)

Computing feature importance via permutation shuffling for 18 features using 5000 rows with 5 shuffle sets...
	2859.91s	= Expected runtime (571.98s per shuffle set)
	1764.66s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
degree,0.148203,0.004344,8.848811e-08,5,0.157147,0.139258
concentrate,0.143120,0.005144,1.999339e-07,5,0.153712,0.132528
average_time_per_learning_session,0.139151,0.003422,4.382885e-08,5,0.146196,0.132106
subscription_duration,0.125801,0.003309,5.739699e-08,5,0.132614,0.118987
recent_learning_achievement,0.118204,0.005941,7.631630e-07,5,0.130436,0.105971
monthly_active_learning_days,0.116160,0.005419,5.669355e-07,5,0.127318,0.105001
recent_login_time,0.115490,0.004742,3.401633e-07,5,0.125253,0.105728
total_completed_courses,0.100236,0.000871,6.832506e-10,5,0.102029,0.098443
payment_pattern,0.093458,0.002937,1.169644e-07,5,0.099506,0.087410
average_login_time,0.088094,0.009111,1.353476e-05,5,0.106853,0.069335


In [37]:
pred_y = predictor.predict(test)
pred_y

0       0
1       0
2       1
3       1
4       1
       ..
9995    0
9996    1
9997    0
9998    0
9999    0
Name: target, Length: 10000, dtype: int64

In [38]:
sub['target'] = pred_y
sub.to_csv('auto_uninter.csv',index=False)

#only uninterest

In [39]:
predictor.feature_importance(data=train)

Computing feature importance via permutation shuffling for 17 features using 5000 rows with 5 shuffle sets...
	2719.94s	= Expected runtime (543.99s per shuffle set)
	1762.67s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
recent_learning_achievement,0.134205,0.002346,1.120677e-08,5,0.139036,0.129373
degree,0.133120,0.004833,2.080929e-07,5,0.143071,0.123169
average_time_per_learning_session,0.122345,0.003946,1.296474e-07,5,0.130469,0.114221
monthly_active_learning_days,0.120429,0.005152,4.010071e-07,5,0.131037,0.109820
average_login_time,0.117965,0.005764,6.816683e-07,5,0.129832,0.106097
concentrate,0.116417,0.004033,1.726170e-07,5,0.124721,0.108112
subscription_duration,0.115340,0.004453,2.661552e-07,5,0.124509,0.106170
recent_login_time,0.112108,0.004142,2.232150e-07,5,0.120636,0.103579
total_completed_courses,0.103912,0.006624,1.970831e-06,5,0.117551,0.090273
payment_pattern,0.092318,0.004093,4.622283e-07,5,0.100745,0.083892


# rise to base_only_pass

In [ ]:
# passionate 75,3 >
# uninterest 5,37 >

In [45]:
train['passionate'] = 0
train.loc[(train['average_time_per_learning_session'] > 103) & (train['abandoned_learning_sessions'] <= 3), 'passionate'] = 1

test['passionate'] = 0
test.loc[(test['average_time_per_learning_session'] > 103) & (test['abandoned_learning_sessions'] <= 3), 'passionate'] = 1

In [ ]:
train['uninterest'] = 0
train.loc[(train['average_time_per_learning_session'] < 37) & (train['abandoned_learning_sessions'] > 5), 'uninterest'] = 1

test['uninterest'] = 0
test.loc[(test['average_time_per_learning_session'] < 37 ) & (test['abandoned_learning_sessions'] > 5), 'uninterest'] = 1

In [46]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   subscription_duration              10000 non-null  int64  
 1   recent_login_time                  10000 non-null  int64  
 2   average_login_time                 10000 non-null  float64
 3   average_time_per_learning_session  10000 non-null  float64
 4   monthly_active_learning_days       10000 non-null  int64  
 5   total_completed_courses            10000 non-null  int64  
 6   recent_learning_achievement        10000 non-null  float64
 7   abandoned_learning_sessions        10000 non-null  int64  
 8   community_engagement_level         10000 non-null  int64  
 9   preferred_difficulty_level         10000 non-null  int64  
 10  subscription_type                  10000 non-null  int64  
 11  customer_inquiry_history           10000 non-null  int6

In [47]:
from autogluon.tabular import TabularDataset, TabularPredictor
train = TabularDataset(train)
test = TabularDataset(test)

In [48]:
fit_parameters = {
    'presets': 'best_quality',
    'num_bag_folds': 7,
    'num_bag_sets': 5,
    'num_stack_levels': 2
}

predictor = TabularPredictor(
    label='target',
    eval_metric='f1_macro',
    sample_weight='balance_weight'
)

predictor.fit(
    train,
    auto_stack=True,
    fit_weighted_ensemble=True,
    presets=fit_parameters,
    num_gpus=1,
    hyperparameter_tune_kwargs='auto'
)

No path specified. Models will be saved in: "AutogluonModels/ag-20231210_110615"
Presets specified: [{'presets': 'best_quality', 'num_bag_folds': 7, 'num_bag_sets': 5, 'num_stack_levels': 2}]
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=7, num_bag_sets=5
Using predefined sample weighting strategy: balance_weight. Evaluation metrics will ignore sample weights, specify weight_evaluation=True to instead report weighted metrics.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231210_110615"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Wed Aug 30 11:19:59 UTC 2023
CPU Count:          8
Memory Avail:       45.56 GB / 51.00 GB (89.3%)
Disk Space Avail:   126.27 GB / 166.77 GB (75.7%)
Train Data Rows:    10000
Train Data Columns: 17
Label Column:       target
AutoGluon infers your prediction

  0%|          | 0/1 [00:00<?, ?it/s]

	Fitting 35 child models (S1F1 - S5F7) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.02%)
Fitted model: LightGBMXT_BAG_L1/T1 ...
	0.5073	 = Validation score   (f1_macro)
	59.18s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: LightGBM_BAG_L1 ...


  0%|          | 0/1 [00:00<?, ?it/s]

	Fitting 35 child models (S1F1 - S5F7) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.02%)
Fitted model: LightGBM_BAG_L1/T1 ...
	0.5237	 = Validation score   (f1_macro)
	48.77s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: RandomForestGini_BAG_L1 ...
	No hyperparameter search space specified for RandomForestGini_BAG_L1. Skipping HPO. Will train one model based on the provided hyperparameters.
Fitted model: RandomForestGini_BAG_L1 ...
	0.4496	 = Validation score   (f1_macro)
	2.9s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: RandomForestEntr_BAG_L1 ...
	No hyperparameter search space specified for RandomForestEntr_BAG_L1. Skipping HPO. Will train one model based on the provided hyperparameters.
Fitted model: RandomForestEntr_BAG_L1 ...
	0.4486	 = Validation score   (f1_macro)
	3.46s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L1 ..

...

Fitted model: NeuralNetFastAI_BAG_L3/d8a71475 ...
	0.5689	 = Validation score   (f1_macro)
	262.29s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: XGBoost_BAG_L3 ...


  0%|          | 0/1 [00:00<?, ?it/s]

	Fitting 35 child models (S1F1 - S5F7) | Fitting with ParallelLocalFoldFittingStrategy (2.0 workers, per: cpus=4, gpus=0, memory=0.05%)
Fitted model: XGBoost_BAG_L3/T1 ...
	0.5737	 = Validation score   (f1_macro)
	83.76s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch_BAG_L3 ...
/usr/local/lib/python3.10/dist-packages/ray/air/config.py:803: UserWarning: Setting a `RunConfig.local_dir` is deprecated and will be removed in the future. If you are not using remote storage,set the `RunConfig.storage_path` instead. Otherwise, set the`RAY_AIR_LOCAL_CACHE_DIR` environment variable to control the local cache location.
  warnings.warn(


+----------------------------------------------------------+
| Configuration for experiment     NeuralNetTorch_BAG_L3   |
+----------------------------------------------------------+
| Search algorithm                 SearchGenerator         |
| Scheduler                        FIFOScheduler           |
| Number of trials                 1                       |
+----------------------------------------------------------+

View detailed results here: /content/AutogluonModels/ag-20231210_110615/models/NeuralNetTorch_BAG_L3



Fitted model: NeuralNetTorch_BAG_L3/7e15d3bb ...
	0.543	 = Validation score   (f1_macro)
	416.27s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBMLarge_BAG_L3 ...
	Fitting 35 child models (S1F1 - S5F7) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.07%)
	0.5668	 = Validation score   (f1_macro)
	55.79s	 = Training   runtime
	0.66s	 = Validation runtime
Fitting model: WeightedEnsemble_L4 ...
	Ensemble Weights: {'LightGBMXT_BAG_L3/T1': 0.265, 'NeuralNetFastAI_BAG_L3/d8a71475': 0.176, 'CatBoost_BAG_L2/T1': 0.162, 'CatBoost_BAG_L1/T1': 0.118, 'CatBoost_BAG_L3/T1': 0.059, 'LightGBMLarge_BAG_L3': 0.059, 'XGBoost_BAG_L2/T1': 0.044, 'XGBoost_BAG_L3/T1': 0.044, 'NeuralNetFastAI_BAG_L1/0aac08da': 0.015, 'XGBoost_BAG_L1/T1': 0.015, 'NeuralNetTorch_BAG_L1/c56aab40': 0.015, 'LightGBMXT_BAG_L2/T1': 0.015, 'ExtraTreesGini_BAG_L3': 0.015}
	0.5812	 = Validation score   (f1_macro)
	17.17s	 = Training   runtime
	0.02s	 = Validation runtim

In [ ]:
predictor.feature_importance(data=train)

In [49]:
pred_y = predictor.predict(test)
pred_y

0       0
1       1
2       1
3       1
4       1
       ..
9995    0
9996    1
9997    0
9998    1
9999    0
Name: target, Length: 10000, dtype: int64

In [50]:
sub['target'] = pred_y
sub.to_csv('auto_only_pass103.csv',index=False)

#motify sample_weight

In [ ]:
train['harder'] = 0
train.loc[(train['average_time_per_learning_session'] > 167), 'harder'] = 1
test['harder'] = 0
test.loc[(test['average_time_per_learning_session'] > 167), 'harder'] = 1

train['degree'] = train['monthly_active_learning_days'] * train['average_time_per_learning_session']
train['concentrate'] = train['average_time_per_learning_session'] - train['average_login_time']

test['degree'] = test['monthly_active_learning_days'] * test['average_time_per_learning_session']
test['concentrate'] = test['average_time_per_learning_session'] - test['average_login_time']

In [56]:
from autogluon.tabular import TabularDataset, TabularPredictor
train = TabularDataset(train)
test = TabularDataset(test)

In [ ]:
fit_parameters = {
    'presets': 'best_quality',
    'num_bag_folds': 7,
    'num_bag_sets': 5,
    'num_stack_levels': 2
}

predictor = TabularPredictor(
    label='target',
    eval_metric='f1_macro',
    sample_weight='average_time_per_learning_session', #수정 or auto_weight
    weight_evaluation = True
)

predictor.fit(
    train,
    auto_stack=True,
    fit_weighted_ensemble=True,
    presets=fit_parameters,
    num_gpus=1,
    hyperparameter_tune_kwargs='auto'
)